In [1]:
# Set the working directory to the parent directory
import sys
sys.path.append('..')
sys.dont_write_bytecode = True

import os
os.chdir("..")

# Import relevant packages
from src.eda import data_info
from src.models import AutoencoderTrainer, AnomalyDetector, AutoencoderTuner
from src.explainability import ShapKernelExplainer

# Import necessary libraries
import shap
import numpy as np
import pandas as pd
import tensorflow as tf
import pickle

In [2]:
baseline_explainer = ShapKernelExplainer(model_type='baseline')

In [ ]:
detector, noise_multiplier = baseline_explainer.version_info_extract(version='202505080012')
explain_with_kernelshap = baseline_explainer.explain_with_kernelshap(detector, noise_multiplier=noise_multiplier)
print("Kernel SHAP explanation completed.")

  1%|▏         | 152/11457 [01:49<2:30:54,  1.25it/s]